In [1]:
import matplotlib.pyplot as plt
import numpyro
from numpyro import handlers
import numpyro.distributions as dist
import numpyro_stein.stein.kernels as kernels
from numpyro.infer.util import Predictive
from numpyro_stein.stein.autoguides import AutoDelta
from numpyro_stein.util import init_with_noise
from numpyro_stein.stein.stein import SVGD
from numpyro_stein.distributions.normal_mixture_distribution import NormalMixture
from numpyro_stein.stein.kernels import SteinKernel
from numpyro.infer import ELBO
from numpyro.infer.util import init_to_value
from tqdm import tqdm
import seaborn as sns
import os
import jax
import jax.numpy as np
from random import randint
import scipy.io

In [2]:
# From http://theoval.cmp.uea.ac.uk/matlab/default.html
data = scipy.io.loadmat('data/benchmarks.mat')

In [3]:
datasets = {k: {'train': {'input': v['x'][0, 0][v['train'][0, 0][13, :] - 1], 'class': (v['t'][0, 0][v['train'][0, 0][13, :] - 1] == 1).astype('float')[:, 0]}, 'test': {'input': v['x'][0, 0][v['test'][0, 0][13, :] - 1], 'class': (v['t'][0, 0][v['test'][0, 0][13, :] - 1] == 1).astype('float')[:, 0]}} for k, v in data.items() if not str.startswith(k, "__") and not k == 'benchmarks' and v['x'][0, 0].shape[0] > 500}

In [4]:
rng_key = jax.random.PRNGKey(randint(0, int(1e6)))
num_iterations = 3000
num_particles = 100

In [5]:
def model(data, classes=None):
    alpha = numpyro.sample('alpha', dist.InverseGamma(concentration=1.0, rate=0.01))
    w = numpyro.sample('w', dist.Normal(loc=np.zeros(data.shape[1]+1), scale=alpha))
    with numpyro.plate('data', data.shape[0]):
        biased_data = np.concatenate((np.ones((data.shape[0],1)), data), axis=1)
        return numpyro.sample('x', dist.Bernoulli(logits=biased_data @ w), obs=classes)

In [6]:
guide = AutoDelta(model, init_strategy=init_with_noise(init_to_value({'x': -10.}), noise_scale=1.0))

In [7]:
def test_accuracy(rng_key, testset, params, num_pred=100):
    def single_test_accuracy(rng_key, testset, params):
        guide_trace = handlers.trace(handlers.substitute(guide, params)).get_trace(testset['input'])
        model_trace = handlers.trace(handlers.replay(handlers.seed(model, rng_key), guide_trace)).get_trace(testset['input'])
        accuracy = np.count_nonzero(model_trace['x']['value'] == testset['class']) / testset['input'].shape[0] * 100
        return accuracy
    accs = []
    for i in range(num_particles):
        ps = {k: param[i] for k, param in params.items()}
        accs.append(jax.vmap(lambda rnk: single_test_accuracy(rnk, testset, ps))(jax.random.split(rng_key, num_pred)))
    return np.mean(np.stack(accs))

In [8]:
for name, dataset in datasets.items():
    print(name)
    svgd = SVGD(model, guide, numpyro.optim.Adagrad(step_size=.05), ELBO(),
            kernels.RBFKernel(), num_stein_particles=num_particles,
            repulsion_temperature=dataset['train']['input'].shape[0] ** -1 * 0.01)
    svgd_state, loss = svgd.run(rng_key, num_iterations, dataset['train']['input'], dataset['train']['class'])
    print(test_accuracy(svgd_state.rng_key, dataset['train'], svgd.get_params(svgd_state)))

banana
SVGD 262.6: 100%|██████████| 3000/3000 [00:10<00:00, 292.19it/s]
50.22365
diabetis
SVGD 226.35: 100%|██████████| 3000/3000 [00:11<00:00, 258.62it/s]
69.63957
german
SVGD 355.4: 100%|██████████| 3000/3000 [00:13<00:00, 226.81it/s]
65.847916
image
SVGD 583.71: 100%|██████████| 3000/3000 [00:14<00:00, 208.16it/s]
71.91128
ringnorm
SVGD 197.0: 100%|██████████| 3000/3000 [00:12<00:00, 241.96it/s]
67.29388
splice
SVGD 655.49: 100%|██████████| 3000/3000 [00:16<00:00, 183.54it/s]
78.565254
twonorm
SVGD 63.506: 100%|██████████| 3000/3000 [00:12<00:00, 233.72it/s]
93.6621
waveform
SVGD 135.58: 100%|██████████| 3000/3000 [00:12<00:00, 238.55it/s]
81.10428
